<a href="https://colab.research.google.com/github/priyankasenapati/Priyanka_Module16_Big-Data/blob/master/Module_16_Bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
# set the environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

spark-3.0.0-preview2-bin-hadoop2.7/
spark-3.0.0-preview2-bin-hadoop2.7/data/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/sample_movielens_rati

In [15]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-03-23 09:43:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.5’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.60MB/s    in 0.2s    

2020-03-23 09:43:51 (3.60 MB/s) - ‘postgresql-42.2.9.jar.5’ saved [914037/914037]



In [0]:
# Start a Sparkse session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
!pip install -q pyspark

EXTRACT

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep="\t", header=True)

In [19]:
# count the number of records
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

In [20]:
# to count number of rows
 df.count()

5033376

TRANSFORM

In [21]:
# Drop null value
df_clean1 = df.dropna()
df_clean1.count()

5033238

In [22]:
# Remove duplicate rows
df_clean2 = df_clean1.distinct()
df_clean2.count()

5033238

In [23]:
#checking data type before transformation 
df_clean2.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [24]:
#Changing the cloumn format
#importing the function 
from pyspark.sql.types import IntegerType,DateType
from pyspark.sql.functions import col

clean_df3 = df_clean2.withColumn("review_date",df_clean2["review_date"].cast(DateType()))
clean_df4 = clean_df3.withColumn("customer_id",clean_df3["customer_id"].cast(IntegerType()))
clean_df5 = clean_df4.withColumn("product_parent",clean_df4["product_parent"].cast(IntegerType()))
clean_df6 = clean_df5.withColumn("star_rating",clean_df5["star_rating"].cast(IntegerType()))
clean_df7= clean_df6.withColumn("helpful_votes",clean_df6["helpful_votes"].cast(IntegerType()))
clean_df_final=clean_df7.withColumn("total_votes",clean_df7["total_votes"].cast(IntegerType()))
clean_df_final.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2611860|R10000VA4KCWGC|B00HMU5VYG|     149855215|Farming Simulator...|     Mobile_Apps|          5|            6|         15|   N|                N|             YESSSSS|Oh my goodness i ...| 2014-01-03|
|         US|    6343669|R10000WAG70YS2|B00FAPF5U0|      46324555|    Candy Crush Saga|     Mobile_Apps|          5|    

In [25]:
# creating a datafrrame for Postgres
# Table 1 : Review id
review_id_df = clean_df_final.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
# review_id check 1
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [26]:
# checking Table 1
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10000VA4KCWGC|    2611860|B00HMU5VYG|     149855215| 2014-01-03|
|R10000WAG70YS2|    6343669|B00FAPF5U0|      46324555| 2014-01-19|
|R100012BO98PAV|   28294136|B008G0HTUU|     219215039| 2012-07-10|
|R10002H1T5W9ZT|   31059135|B009UX2YAC|     879319057| 2015-08-09|
|R1000382O45PDR|   50148247|B005LIX3T6|     287709243| 2013-05-09|
|R10003HA48BDHT|   23830040|B00B2V66VS|     148127986| 2013-01-26|
|R10003UGGTHTDR|   16265036|B00HGJ18C2|     326848133| 2014-08-05|
|R100044HWNQF7K|   34911252|B00AMOJB2M|     511532915| 2014-09-04|
|R100045LEFQSV4|   44992645|B00785P2QC|     162354237| 2014-01-24|
|R100048KBNBB6N|   52828389|B0050DZN4K|     845697367| 2014-08-23|
|R10006X6BFWY6O|   41050840|B004VMVF5C|      61146541| 2013-01-15|
|R10007LRVPLNSV|   51983368|B0081JPTCQ|     657075255| 2014-10

In [0]:
#creating product table using distinct values only 
from pyspark.sql.functions import col, countDistinct
distinct_value=clean_df_final.distinct()
product_df=distinct_value.select(["product_id","product_title"])
#droping duplicate value 
uniquevalue= product_df.dropDuplicates()
#product_df.select(countDistinct("product_id")).show()

In [28]:
#checking distinct 
uniquevalue.count()

127060

In [29]:
uniquevalue.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HMU5VYG|Farming Simulator...|
|B00FAPF5U0|    Candy Crush Saga|
|B008G0HTUU|      Cartoon Wars 2|
|B009UX2YAC|      Subway Surfers|
|B005LIX3T6|Weaving Wings Gui...|
|B00B2V66VS|        Temple Run 2|
|B00HGJ18C2|           Solitaire|
|B00AMOJB2M|   Pyramid Spirits 3|
|B00785P2QC|Virtual City Play...|
|B0050DZN4K|Feedly Reader | B...|
|B004VMVF5C|Aldiko Book Reade...|
|B0081JPTCQ|      Handy Note Pro|
|B00TSE6VBQ|Village Life: Lov...|
|B0094BB4TW|            Facebook|
|B007L3M1W4|    Virtual Makeover|
|B00ANUMR1W|       DRAGON SLAYER|
|B004AZH4C8|        Enjoy Sudoku|
|B00FGQCP6A|Hidden Objects:  ...|
|B008XG1X18|           Pinterest|
|B004RJMUJO|         Fruit Ninja|
+----------+--------------------+
only showing top 20 rows



In [30]:
#To check unique value 
product_df.select(countDistinct("product_id")).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    127060|
+--------------------------+



In [31]:
#creating Customer_df
customers=clean_df_final.groupby("customer_id").count()
customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    2611860|  103|
|    6343669|    2|
|   28294136|    3|
|   31059135|   43|
|   50148247|   13|
|   23830040|   26|
|   16265036|    8|
|   34911252|   11|
|   44992645|    1|
|   52828389|    5|
|   41050840|   24|
|   51983368|    2|
|    1093452|    1|
|   17436078|    9|
|   48450902|   14|
|     911024|    4|
|   12212931|    1|
|   50149493|    8|
|   46255423|    1|
|   16426258|    6|
+-----------+-----+
only showing top 20 rows



In [33]:
from pyspark.sql.functions import col

customers_df = customers.select(col("customer_id").alias("customer_id"), col("count").alias("customer_count"))
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    2611860|           103|
|    6343669|             2|
|   28294136|             3|
|   31059135|            43|
|   50148247|            13|
|   23830040|            26|
|   16265036|             8|
|   34911252|            11|
|   44992645|             1|
|   52828389|             5|
|   41050840|            24|
|   51983368|             2|
|    1093452|             1|
|   17436078|             9|
|   48450902|            14|
|     911024|             4|
|   12212931|             1|
|   50149493|             8|
|   46255423|             1|
|   16426258|             6|
+-----------+--------------+
only showing top 20 rows



In [34]:
#creating Vine table

vine_df=clean_df_final.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10000VA4KCWGC|          5|            6|         15|   N|
|R10000WAG70YS2|          5|            0|          0|   N|
|R100012BO98PAV|          4|            0|          0|   N|
|R10002H1T5W9ZT|          4|            0|          0|   N|
|R1000382O45PDR|          5|            1|          1|   N|
|R10003HA48BDHT|          5|            9|         10|   N|
|R10003UGGTHTDR|          4|            0|          0|   N|
|R100044HWNQF7K|          5|            0|          0|   N|
|R100045LEFQSV4|          1|            2|          3|   N|
|R100048KBNBB6N|          1|            0|          0|   N|
|R10006X6BFWY6O|          5|            2|          2|   N|
|R10007LRVPLNSV|          4|            0|          0|   N|
|R10008LFASI2DR|          4|            0|          0|   N|
|R100099JS210W9|          4|            

LOAD


In [35]:
# Configure settings for RDS
#for Postgrate driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
mode = "append"
jdbc_url="jdbc:postgresql://amazon-reviews.cl9q6ilmivtn.us-east-2.rds.amazonaws.com:5432/amazon reviews"
config = {"user":"postgres",
          "password": "MIMI1993",
          "driver":"org.postgresql.Driver"}

--2020-03-23 10:04:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.6’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.62MB/s    in 0.2s    

2020-03-23 10:04:17 (3.62 MB/s) - ‘postgresql-42.2.9.jar.6’ saved [914037/914037]



In [36]:
print(jdbc_url)
print(config)

jdbc:postgresql://amazon-reviews.cl9q6ilmivtn.us-east-2.rds.amazonaws.com:5432/amazon reviews
{'user': 'postgres', 'password': 'MIMI1993', 'driver': 'org.postgresql.Driver'}


In [37]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
review_id_1=review_id_df.dropna()

In [0]:
# Write DataFrame to review_id_table table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write DataFrame to products table in RDS
uniquevalue.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [0]:
# Write DataFrame to vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
